本文件是linear regression在实现方面的细节，具体理论参见我的博客。

## 数据准备

In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [2]:
data=load_boston()
X=data.data
Y=data.target

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

数据$X$是一个$(n{\times}m)$的矩阵，每一行是一个样本，每一列代表一个特征：

In [3]:
n=X_train.shape[0]
m=X_train.shape[1]

标签$Y$是一个列向量，其行数与$X$相同：

In [4]:
Y_train = Y_train.reshape((n, 1))
Y_test = Y_test.reshape((-1, 1))

## 粗略模型

模型表达式为：
$$
\hat{Y}=XW^{T}+b
$$
其中权重系数$W$的形状为$(1,m)$，偏置系数$b$为单变量系数。

In [5]:
W = np.random.rand(m).reshape((1, -1))  # 权重，行向量
b = np.ones((1, 1))  # 偏置

Y_hat=np.dot(X_train, W.T)+b

模型的损失函数为：
$$
\begin{align}
L&=\sum\limits_{i=1}^n(y^{(i)}-\hat{y}^{(i)})^{2} \\
&=\frac{1}{n}(Y-\hat{Y})^{T}(Y-\hat{Y}) \\
\end{align}
$$
损失函数关于参数$W$与$b$的梯度可以求得：
$$
\begin{align}
\frac{\partial{L}}{\partial{W}}&=\frac{2}{n}(\hat{Y}-Y)^{T}{\cdot}X \\
\frac{\partial{L}}{\partial{b}}&=\frac{2}{n}(\hat{Y}-Y)^{T}{\cdot}[1,1,...,1]^{T} \\
\end{align}
$$

In [6]:
dW = 2 * (Y_hat - Y_train).T.dot(X_train) / n
db = 2 * (Y_hat - Y_train).T.dot(np.ones((n, 1))) / n

参数的迭代更新公式：
$$
W:=W-{\alpha}\frac{\partial{L}}{\partial{W}}, \quad b:b-{\alpha}\frac{\partial{L}}{\partial{b}}
$$

In [7]:
max_iter=2000
alpha=0.000001        # 注意学习率过大会导致震荡，然后误差越来越大

for i in range(max_iter+1):
    Y_hat=np.dot(X_train, W.T)+b
    
    dW = 2 * (Y_hat - Y_train).T.dot(X_train) / n
    db = 2 * (Y_hat - Y_train).T.dot(np.ones((n, 1))) / n
    
    W = W - alpha * dW
    b = b - alpha * db

使用该模型分别对训练集与预测集做预测：

In [8]:
Y_pred_train=np.dot(X_train, W.T) + b
Y_pred_test=np.dot(X_test, W.T) + b

定义一个RMSE损失函数来评价模型的表现：

In [9]:
def RMSE(Y_true,Y_pred):
    return sum((Y_true-Y_pred)**2)**0.5

print(RMSE(Y_train,Y_pred_train),RMSE(Y_test,Y_pred_test))

[209.03039228] [105.01153698]


模型简单打包：

In [25]:
def linear_reg(X,Y,alpha=0.000001,max_iter=2000):
    n=X.shape[0]
    m=X.shape[1]
    
    W = np.random.rand(m).reshape((1, -1))  # 权重，行向量
    b = np.ones((1, 1))  # 偏置

    for i in range(max_iter+1):
        Y_hat=np.dot(X, W.T)+b

        dW = 2 * (Y_hat - Y).T.dot(X) / n
        db = 2 * (Y_hat - Y).T.dot(np.ones((n, 1))) / n

        W = W - alpha * dW
        b = b - alpha * db
        
        if i%200==0:
            Y_hat=np.dot(X, W.T)+b
            L=sum((Y-Y_hat)**2)**0.5
            print(L,end='\t')

    return W,b

W,b=linear_reg(X_train,Y_train)

[3096.84571667]	[219.1522414]	[211.36205226]	[205.07451308]	[199.96630972]	[195.78432519]	[192.32949228]	[189.44514806]	[187.00819393]	[184.92215757]	[183.11165466]	

## 数据归一化
**Normalization：**
$$
x=\frac{x-x_{min}}{x_{max}-x_{min}}
$$

In [11]:
X=np.row_stack((X_train,X_test))

X_max=X.max(axis=0)
X_min=X.min(axis=0)

X_train_norm=(X_train-X_min)/(X_max-X_min)
X_test_norm=(X_test-X_min)/(X_max-X_min)

对数据归一化之后再测试模型表现：

In [27]:
W,b=linear_reg(X_train_norm,Y_train)

[417.92167284]	[417.46312635]	[417.00543837]	[416.54860759]	[416.09263273]	[415.63751251]	[415.18324563]	[414.72983082]	[414.27726679]	[413.82555226]	[413.37468596]	

因为数据做了归一化，整个数据集上的梯度分布得到了改良，所以可以调大学习率，由此可以看出数据标准化在线性回归上的威力：

In [13]:
W,b=linear_reg(X_train_norm,Y_train,alpha=0.1)

[229.5317887]	[102.93234471]	[97.0599989]	[95.47338681]	[94.67825928]	[94.1663453]	[93.81118957]	[93.55803605]	[93.37505064]	[93.24157706]	[93.14358568]	

**Standardization：**
$$
x=\frac{x-x_{\mu}}{\sigma}
$$

In [14]:
X=np.row_stack((X_train,X_test))

X_avg=X.mean(axis=0)
X_std=X.std(axis=0)

X_train_std=(X_train-X_avg)/X_std
X_test_std=(X_test-X_avg)/X_std

In [31]:
W,b=linear_reg(X_train_std,Y_train)

[476.48493421]	[476.19868728]	[475.91303235]	[475.62796719]	[475.34348955]	[475.05959722]	[474.77628798]	[474.49355962]	[474.21140994]	[473.92983674]	[473.64883785]	

In [16]:
W,b=linear_reg(X_train_std,Y_train,alpha=0.1)

[370.32667861]	[92.8702853]	[92.86133807]	[92.86127362]	[92.86127315]	[92.86127315]	[92.86127315]	[92.86127315]	[92.86127315]	[92.86127315]	[92.86127315]	

这个简单实验比较发现，相比于Normalization，Standardization能够更快地加速模型的收敛。

数据归一化工具简单打包：

In [17]:
def Standardization(X_train,X_test):
    X=np.row_stack((X_train,X_test))

    X_avg=X.mean(axis=0)
    X_std=X.std(axis=0)

    X_train_std=(X_train-X_avg)/X_std
    X_test_std=(X_test-X_avg)/X_std
    return X_train_std,X_test_std

## mini-batch梯度下降

In [63]:
def linear_reg(X,Y,alpha=0.000001,max_iter=2000,batch_size=99999):
    n=X.shape[0]
    m=X.shape[1]
    num_batch = n // batch_size
    
    W = np.random.rand(m).reshape((1, -1))  # 权重，行向量
    b = np.ones((1, 1))  # 偏置

    for epoch in range(max_iter+1):
        
        ######  mini-batch  ######
        for i in range(num_batch + 1):
            start_index = i * batch_size
            end_index = (i + 1) * batch_size
            if end_index <= n:
                X_batch = X[start_index:end_index + 1]
                Y_batch = Y[start_index:end_index + 1]
            else:
                X_batch = X[start_index:]
                Y_batch = Y[start_index:]
                
        n_batch=X_batch.shape[0]
        Y_hat_batch=np.dot(X_batch, W.T)+b

        dW = 2 * (Y_hat_batch - Y_batch).T.dot(X_batch) / n_batch
        db = 2 * (Y_hat_batch - Y_batch).T.dot(np.ones((n_batch, 1))) / n_batch
        ######  mini-batch  ######
        
        W = W - alpha * dW
        b = b - alpha * db
        
        if epoch%200==0:
            Y_hat=np.dot(X, W.T)+b
            L=sum((Y-Y_hat)**2)**0.5
            print(L,end='\t')

    return W,b

W,b=linear_reg(X_train,Y_train,batch_size=32)

[4054.64707425]	[266.96807985]	[253.24645327]	[245.19175745]	[240.23696832]	[236.93782462]	[234.51337405]	[232.55169565]	[230.83919826]	[229.26669286]	[227.77916272]	

单纯的mini-batch并没有很明显的提升模型表现，我们再加上Standardization：

In [90]:
X_train_std,X_test_std=Standardization(X_train,X_test)
W,b=linear_reg(X_train_std,Y_train,alpha=0.001,batch_size=32)

[473.20273442]	[338.15078566]	[281.41435524]	[250.28595535]	[231.39037269]	[218.79341172]	[209.82423767]	[203.15932313]	[198.05613147]	[194.05332841]	[190.84553224]	

额，表现并不是很理，甚至在加大max_iter值后模型还是没有收敛，可能是数据量太小，mini-batch引入的随机性对模型的收敛起了一个反作用。

In [19]:
# class LinearRegression:
#     def __init__(self, lr=0.00001, batch_size=32, max_iter=1000):
#         self.lr = lr
#         self.batch_size = batch_size
#         self.max_iter = max_iter
#         self.W = None
#         self.b = None

#     def fit(self, X, Y):
#         X = X.copy()
#         Y = Y.copy()

#         n = X.shape[0]  # 样本数
#         m = X.shape[1]  # 特征数
#         assert Y.shape[0] == n  # 数据与标签应该相等
#         Y = Y.reshape((n, 1))  # 标签，列向量

#         self.W = np.random.rand(m).reshape((1, -1))  # 权重，行向量
#         self.b = np.ones((1, 1))  # 偏置

#         assert Y.shape == (n, 1)

#         num_batch = n // self.batch_size

#         for epoch in range(self.max_iter):
#             for i in range(num_batch + 1):
#                 start_index = i * self.batch_size
#                 end_index = (i + 1) * self.batch_size
#                 if end_index <= n:
#                     X_batch = X[start_index:end_index + 1]
#                     Y_batch = Y[start_index:end_index + 1]
#                 else:
#                     X_batch = X[start_index:]
#                     Y_batch = Y[start_index:]

#                 Y_hat = X_batch.dot(self.W.T) + self.b
#                 dW = 2 * (Y_hat - Y_batch).T.dot(X_batch) / n
#                 db = 2 * (Y_hat - Y_batch).T.dot(np.ones((X_batch.shape[0], 1))) / n
#                 assert (dW.shape == self.W.shape) & (db.shape == self.b.shape)

#                 self.W = self.W - self.lr * dW
#                 self.b = self.b - self.lr * db

#     def predict(self, X):
#         X = X.copy()
#         return np.squeeze(np.dot(X, self.W.T) + self.b)        # 将矩阵压缩成向量，与原始输入Y保持一致

    
# line_reg=LinearRegression()
# line_reg.fit(X_train,Y_train)

# def RMSE(Y_true,Y_pred):
#     return sum((Y_true-Y_pred)**2)**0.5

# Y_pred=line_reg.predict(X_test)
# RMSE(Y_test,Y_pred)